# Evolutionary Camouflage Versus a Learning Predator Population

---

EvoCamoVsLearnPredPop.ipynb

Started as a copy of EvoCamoVsLearningPredator.ipynb as of 20220607

On 20220716 starting from here to prototype “local” version running on my Apple Silicon M1 laptop. Got as far as trying to read pre-tained “FCD6_rc4” model but it complained it was compiled for Intel, while this is Apple Silicon.

OK now on 20220722 I am converting this back to "Rube Goldberg" mode where predator vision happens in the cloud, and camouflage evolution runs on my laptop. I tried using the [TensorFlow-Metal plugin](https://developer.apple.com/metal/tensorflow-plugin/), but ran into a [bug](https://developer.apple.com/forums/thread/706920). Until that is fixed I will fall back to the old half-cloud-half-laptop approach.

In [1]:
# "Rube Goldberg" mode refers to running camouflage evolution on my laptop while
# running predator vision in cloud via Colab. State is passed back and forth via
# files on Google Drive.
Rube_Goldberg_mode = True
def if_RG_mode(for_RG_mode, for_normal_mode):
    return for_RG_mode if Rube_Goldberg_mode else for_normal_mode

# PredatorEye directory on Drive.
pe_directory = '/content/drive/My Drive/PredatorEye/'

# Shared "communication" directory on Drive.
shared_directory = if_RG_mode(pe_directory + 'evo_camo_vs_static_fcd/',
                              '/Users/cwr/comms/')

# This was meant (20220716) to allow reading original pre-trained model from
# Google Drive, but I'll need to retrain it for M1 (Apple Silicon).
g_drive_pe_dir = ('/Users/cwr/Library/CloudStorage/' +
                  'GoogleDrive-craig.w.reynolds@gmail.com/' +
                  'My Drive/PredatorEye/')

# Directory for pre-trained Keras/TensorFlow models.
saved_model_directory = if_RG_mode(pe_directory, g_drive_pe_dir) + 'saved_models/'


print('Rube_Goldberg_mode =', Rube_Goldberg_mode)
print('shared_directory =', shared_directory)
print('saved_model_directory =', saved_model_directory)

# Pathname of pre-trained Keras/TensorFlow model
# trained_model = saved_model_directory + '20220202_1211_Find_3_Disks_complex'
# trained_model = saved_model_directory + '20220222_1747_F3D_augmented_rc4'
# trained_model = saved_model_directory + '20220227_0746_F3D2_a'
# trained_model = saved_model_directory + '20220304_1135_FCD5_a'
trained_model = saved_model_directory + '20220321_1711_FCD6_rc4'
# model = []

# Directory on Drive for storing fine-tuning dataset.
fine_tuning_directory = shared_directory + 'fine_tuning/'

my_prefix = "find_"
other_prefix = "camo_"

my_suffix =  ".txt"
# other_suffix = ".jpeg"
other_suffix = ".png"

fcd_image_size = 1024
fcd_disk_size = 201

import time
import PIL
from pathlib import Path

from tensorflow import keras
import numpy as np
import random

# %tensorflow_version 2.x
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

from tensorflow.keras import backend as keras_backend
keras_backend.set_image_data_format('channels_last')

# Import DiskFind utilities for PredatorEye.
import sys
if Rube_Goldberg_mode:
    sys.path.append('/content/drive/My Drive/PredatorEye/shared_code/')
else:
    sys.path.append('/Users/cwr/Documents/code/PredatorEye/')
import DiskFind as df

Rube_Goldberg_mode = True
shared_directory = /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/
saved_model_directory = /content/drive/My Drive/PredatorEye/saved_models/
TensorFlow version: 2.8.2


# Ad hoc “predator server”

In [2]:
# Top level: wait for camo_xxx.jpeg files to appear, respond with find_xxx.txt
def start_run(step = 0):
    if step == 0:
        print('Start run in', shared_directory )
    else:
        print('Continue run at step', step, ' in', shared_directory)
    while True:
        perform_step(step, shared_directory)
        step += 1

# Continue from from the last camo_xxx.jpeg file.
def restart_run():
    start_run(newest_file_from_other(shared_directory))

# Single step: wait for camo file, write response, delete previous response.
def perform_step(step, directory):
    wait_for_reply(step, shared_directory)
    write_response_file(step, shared_directory)
    delete_find_file(step - 1, shared_directory)

# Read image file for step, apply pre-trained model, write response file.
def write_response_file(step, directory):
    # Read image file and check for expected format.
    image_pathname = make_camo_pathname(step, directory)
    pixel_tensor = df.read_image_file_as_pixel_tensor(image_pathname)
    assert df.check_pixel_tensor(pixel_tensor), ('wrong file format: ' +
                                                 image_pathname)
    ############################################################################
    # TODO 20220729 prototype storing what had been the global predator model
    # with one stored in Predator class.

    # model = Predator.get_population()[0]

    # model = Predator.population[0]

    # default_predator = population_of_predators[0]

    # default_predator = Predator.population[0]
    # model = default_predator.model

    model = Predator.default_predator().model

    # Run pre-trained model on new image.
    prediction = model.predict(tf.convert_to_tensor([pixel_tensor]))[0]
    ############################################################################
    # Generate response file.
    response_string = str(prediction[0]) + " " + str(prediction[1])
    verify_comms_directory_reachable()
    with open(make_find_pathname(step, directory), 'w') as file:
        file.write(response_string)
    print('Wrote ' + "'" + response_string + "'",
          'to response file', Path(make_find_pathname(step, directory)).name)
    ############################################################################
    # TODO 20220729 prototype storing what had been the global predator model
    # with one stored in Predator class.

    # TODO 20220730 purely for testing -- delete me
    opm = Predator.second_predator().model
    print('"other" prediction:',
          opm.predict(tf.convert_to_tensor([pixel_tensor]))[0])

    # Predator learns from recent experience.
    # fine_tune_predator(pixel_tensor, prediction, step, directory)
    # default_predator.fine_tune_predator(pixel_tensor, prediction, step, directory)
    Predator.default_predator().fine_tune_predator(pixel_tensor, prediction,
                                                   step, directory)
    ############################################################################

# Delete the given file, usually after having written the next one.
def delete_find_file(step, directory):
    # Why doesn't pathlib provide a Path.remove() method like os?
    # TODO oh, missing_ok was added at pathlib version 3.8.
    # Path(makeMyPathname(step, directory)).unlink(missing_ok=True)
    p = Path(make_find_pathname(step, directory))
    if p.exists():
        p.unlink()

# Delete any remaining file in commuications directory to start a new run.
def clean_up_communication_directory():
    def delete_directory_contents(directory_path):
        for path in directory_path.iterdir():
            print('Removing from communication directory:', path)
            if path.is_dir():
                delete_directory_contents(path)
                path.rmdir()
            else:
                path.unlink()
    delete_directory_contents(Path(shared_directory))

# From pathname for file of given step number from the "other" agent.
def make_camo_pathname(step, directory):
    return directory + other_prefix + str(step) + other_suffix

# Form pathname for "find_xx.txt" response file from "this" agent.
def make_find_pathname(step, directory):
    return directory + my_prefix + str(step) + my_suffix

# Form pathname for "prey_xx.txt" ground truth file from "other" agent.
def make_prey_pathname(step, directory):
    return directory + 'prey_' + str(step) + '.txt'

# Used to ping the comms directory when it seems hung.
def write_ping_file(count, step, directory):
    pn = directory + 'ping_cloud_' + str(step) + '.txt'
    verify_comms_directory_reachable()
    with open(pn, 'w') as file:
        file.write(str(count))
    print('Ping comms: ', count, pn)

# Wait until other agent's file for given step appears.
def wait_for_reply(step, directory):
    camo_pathname = Path(make_camo_pathname(step, directory))
    camo_filename = camo_pathname.name
    prey_pathname = Path(make_prey_pathname(step, directory))
    prey_filename = prey_pathname.name
    print('Waiting for', camo_filename, 'and', prey_filename, '...',
          end='', flush=True)
    start_time = time.time()
    # Loop until both files are present, waiting 1 second between tests.
    test_count = 0
    while not (is_file_present(camo_pathname) and
               is_file_present(prey_pathname)):
        time.sleep(1)
        test_count += 1
        if (test_count % 100) == 0:
            write_ping_file(test_count, step, directory)
    print(' done, elapsed time:', int(time.time() - start_time), 'seconds.')

# Like fs::exists()
def is_file_present(file):
    result = False
    verify_comms_directory_reachable()
    filename = Path(file).name
    directory = Path(file).parent
    for i in directory.iterdir():
        if i.name == filename:
            result = True
    return result

# Returns the step number of the newest file from "other" in given directory.
# (So if "camo_573.jpeg" is the only "other" file there, returns int 573)
def newest_file_from_other(directory):
    steps = [0]  # Default to zero in case dir is empty.
    for filename in Path(directory).iterdir():
        name = filename.name
        if other_prefix == name[0:len(other_prefix)]:
            steps.append(int(name.split(".")[0].split("_")[1]))
    return max(steps)

################################################################################
# TODO 20220728 maybe fine_tune_images, fine_tune_labels, etc. should be
# associated with Predator class, but not instance, like c++ class static member.

# # Accumulated a new “training set” of the most recent N steps seen so far. (See
# # https://cwreynolds.github.io/TexSyn/#20220421 and ...#20220424 for discussion
# # of this parameter. Had been 1, then 100, then 200, then finally, infinity.) 
# # max_training_set_size = float('inf') # keep ALL steps in training set, use GPU.
# max_training_set_size = 500 # Try smaller again, "yellow flowers" keeps failing.
# # List of "pixel tensors".
# fine_tune_images = []
# # List of xy3 [[x,y],[x,y],[x,y]] for 3 prey centers.
# fine_tune_labels = []

# # Keep track of how often selected prey is nearest center:
# nearest_center = 0

################################################################################

################################################################################
# TODO 20220728 fine_tune_predator() should become a method of Predator class.

# # Apply fine-tuning to (originally pre-trained) predator. Use recent steps as
# # training set. Assume they were "near misses" and so training label is actual
# # (ground truth) center of disk nearest prediction. Keep a max number of old
# # steps to allow gradually forgetting the earliest part of the run.
# def fine_tune_predator(pixel_tensor, prediction, step, directory):
#     # Assume the predator was "aiming for" that one but missed by a bit.
#     xy3 = read_3_centers_from_file(step, directory)
#     sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)

#     # Accumulate the most recent "max_training_set_size" training samples.
#     global fine_tune_images
#     global fine_tune_labels
#     fine_tune_images.append(pixel_tensor)
#     fine_tune_labels.append(sorted_xy3)

#     # If training set has become too large, slice off first element of each.
#     if len(fine_tune_images) > max_training_set_size:
#         fine_tune_images = fine_tune_images[1:]
#         fine_tune_labels = fine_tune_labels[1:]

#     # keep track of how often selected prey is nearest center:
#     temp = xy3.copy()  # needed?
#     sorted_by_dist_to_center = sort_xy3_by_proximity_to_point(temp, [0.5, 0.5])
#     if sorted_by_dist_to_center[0] == sorted_xy3[0]:
#         global nearest_center
#         nearest_center += 1
#     print('  nearest_center:',
#           str(int(100 * float(nearest_center) / (step + 1))) + '%',
#           '(nearest_center =', nearest_center, ', steps =', step + 1, ')')

#     # Convert training data list to np arrays
#     images_array = np.array(fine_tune_images)
#     labels_array = np.array([x[0] for x in fine_tune_labels])
#     print('images_array.shape =', images_array.shape)
#     print('labels_array.shape =', labels_array.shape)

# 	# skip fine-tuning until dataset is large enough (10% of max size).
#     if images_array.shape[0] > (max_training_set_size * 0.1):
#         # Do fine-tuning training step using data accumulated during run.
#         history = model.fit(x=images_array, y=labels_array)
#         # Keep log of in_disk metric:
#         write_in_disk_log(step, history)

################################################################################

# Read ground truth prey center location data provided in "prey_n.txt" file.
def read_3_centers_from_file(step, directory):
    # Read contents of file as string.
    verify_comms_directory_reachable()
    with open(make_prey_pathname(step, directory), 'r') as file:
        prey_centers_string = file.read()
    # Split string at whitespace, map to 6 floats, reshape into 3 xy pairs.
    # (TODO could probably be rewritten cleaner with "list comprehension")
    array = np.reshape(list(map(float, prey_centers_string.split())), (3, 2))
    return array.tolist()

# Keep log of in_disk metric.
def write_in_disk_log(step, history):
    if step % 10 == 0:
        in_disk = history.history["in_disk"][0]
        pathname = shared_directory + 'in_disk_log.csv'
        verify_comms_directory_reachable()
        with open(pathname, 'a') as file:
            if step == 0:
                file.write('step,in_disk\n')
            file.write(str(step) + ',' + "{:.4f}".format(in_disk) + '\n')

# Just wait in retry loop if shared "comms" directory become unreachable.
# Probably will return shortly, better to wait than signal a file error.
# (This is called from places with a local "directory" but it uses global value.)
def verify_comms_directory_reachable():
    seconds = 0
    # shared_directory_pathname = Path(shared_directory)
    # while not shared_directory_pathname.is_dir():
    while not Path(shared_directory).is_dir():
        print("Shared “comms” directory,", shared_directory, 
              "has been inaccessible for", seconds, "seconds.")
        time.sleep(1)  # wait 1 sec
        seconds += 1

# Given 3 prey positions ("xy3"), sort them by proximity to "point" (prediction)
def sort_xy3_by_proximity_to_point(xy3, point):
    # print('xy3 =', xy3)
    xy3_plus_distance = [[df.dist2d(xy, point), xy] for xy in xy3]
    # print('xy3_plus_distance =', xy3_plus_distance)
    sorted_xy3_plus_key = sorted(xy3_plus_distance, key=lambda x: x[0])
    # print('sorted_xy3_plus_key =', sorted_xy3_plus_key)
    sorted_xy3 = [x[1] for x in sorted_xy3_plus_key]
    # print('sorted_xy3 =', sorted_xy3)
    return sorted_xy3

# Read pre-trained model

As I integrate this into the Predator class, this is no longer “Read pre-trained model” but more like “Some utilities for reading the pre-trained model”

In [3]:
# Read pre-trained TensorFlow "predator vision" model.

# print('Reading pre-trained model from:', trained_model)

# ad hoc workaround suggested on https://stackoverflow.com/q/66408995/1991373
#
# dependencies = {
#     'hamming_loss': tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"),
#     'attention': attention(return_sequences=True)
# }
#
# dependencies = {
#     'valid_accuracy': ValidAccuracy
# }

# Calculates RELATIVE disk radius on the fly -- rewrite later.
def fcd_disk_radius():
    return (float(fcd_disk_size) / float(fcd_image_size)) / 2

# Given two tensors of 2d point coordinates, return a tensor of the Cartesian
# distance between corresponding points in the input tensors.
def corresponding_distances(y_true, y_pred):
    true_pos_x, true_pos_y = tf.split(y_true, num_or_size_splits=2, axis=1)
    pred_pos_x, pred_pos_y = tf.split(y_pred, num_or_size_splits=2, axis=1)
    dx = true_pos_x - pred_pos_x
    dy = true_pos_y - pred_pos_y
    distances = tf.sqrt(tf.square(dx) + tf.square(dy))
    return distances

# 20211231 copied from Find_Concpocuous_Disk
def in_disk(y_true, y_pred):
    distances = corresponding_distances(y_true, y_pred)
    # relative_disk_radius = (float(fcd_disk_size) / float(fcd_image_size)) / 2

    # From https://stackoverflow.com/a/42450565/1991373
    # Boolean tensor marking where distances are less than relative_disk_radius.
    # insides = tf.less(distances, relative_disk_radius)
    insides = tf.less(distances, fcd_disk_radius())
    map_to_zero_or_one = tf.cast(insides, tf.int32)
    return map_to_zero_or_one

dependencies = { 'in_disk': in_disk }

################################################################################
# TODO 20220726 change for Predator class.

# model = keras.models.load_model(trained_model, custom_objects=dependencies)

def read_default_pre_trained_model():
    print('Reading pre-trained model from:', trained_model)
    return keras.models.load_model(trained_model, custom_objects=dependencies)

################################################################################

# Predator class

In [4]:
################################################################################
# TODO 20220729 having trouble defining this as a "class variable" (c++ static)
# so moving it out here for now.

# population_of_predators = []

################################################################################


# TODO switch to using Python standard """doc strings""" for methods below?

################################################################################
# TODO 20220724 define for Predator, we want to have a population of them.

class Predator:
    """Represents a Predator in the camouflage simulation. It has a CNN-based
       model of visual hunting that identified the position of likely prey."""

    ############################################################################
    # TODO 20220729 prototype storing what had been the global predator model
    # with one stored in Predator class.
    # untested

    population = []

    # # Static class function, returns global list of all Predators.
    # @ staticmethod
    # def get_population():
    #     print('in get_population(), size is', len(self.population))
    #     return self.population

    # # Add or remove a Predator instance from the global population.
    # # def add_to_population(self):
    # #     population.append(self)
    # def remove_from_population(self):
    #     population.remove(self)
    ############################################################################


    def __init__(self):
        # Each Predator has its own copy of a prey-finding CNN-based model.
        self.model = []
        ########################################################################
        # TODO 20220729 
        # population.append(self)
        # self.population.append(self)
        # global population_of_predators
        # population_of_predators.append(self)
        global population_of_predators
        self.population.append(self)
        ########################################################################

    def initialize_to_pre_trained_model(self):
        self.model = read_default_pre_trained_model()

    ############################################################################
    # TODO 20220730 rather than define this concept elsewhere, move it inside,
    #               even if it is transitional/experimental.
    # def default_predator(self):
    #     return self.population[0]
    # def default_predator():
    #     return self.population[0]
    # def default_predator():
    #     return population[0]

    def default_predator():
        return Predator.population[0]
    
    def second_predator():
        return Predator.population[1]
    
    ############################################################################

    ############################################################################
    # TODO 20220728 maybe fine_tune_images, fine_tune_labels, etc. should be
    # associated with Predator class, but not instance, like c++ class static member.
    #
    # Arguably, this managing of the dataset for fine tuning ought to be inside
    # its own class. But for now it will just be inline class attributes of Predator. 

    # Accumulated a new “training set” of the most recent N steps seen so far. (See
    # https://cwreynolds.github.io/TexSyn/#20220421 and ...#20220424 for discussion
    # of this parameter. Had been 1, then 100, then 200, then finally, infinity.) 
    # max_training_set_size = float('inf') # keep ALL steps in training set, use GPU.
    max_training_set_size = 500 # Try smaller again, "yellow flowers" keeps failing.
    # List of "pixel tensors".
    fine_tune_images = []
    # List of xy3 [[x,y],[x,y],[x,y]] for 3 prey centers.
    fine_tune_labels = []

    # Keep track of how often selected prey is nearest center:
    nearest_center = 0

    ############################################################################

    ############################################################################
    # TODO 20220728 fine_tune_predator() should become a method of Predator class.
    # Maybe it should be called just fine_tune(), Predator.fine_tune()

    # Apply fine-tuning to (originally pre-trained) predator. Use recent steps as
    # training set. Assume they were "near misses" and so training label is actual
    # (ground truth) center of disk nearest prediction. Keep a max number of old
    # steps to allow gradually forgetting the earliest part of the run.
    # def fine_tune_predator(pixel_tensor, prediction, step, directory):
    def fine_tune_predator(self, pixel_tensor, prediction, step, directory):
        # Assume the predator was "aiming for" that one but missed by a bit.
        xy3 = read_3_centers_from_file(step, directory)
        sorted_xy3 = sort_xy3_by_proximity_to_point(xy3, prediction)

        # Accumulate the most recent "max_training_set_size" training samples.
        ########################################################################
        # TODO 20220729
        # global fine_tune_images
        # global fine_tune_labels
        # fine_tune_images.append(pixel_tensor)
        # fine_tune_labels.append(sorted_xy3)
        self.fine_tune_images.append(pixel_tensor)
        self.fine_tune_labels.append(sorted_xy3)
        ########################################################################

        # If training set has become too large, slice off first element of each.
        ########################################################################
        # TODO 20220729
        # if len(fine_tune_images) > max_training_set_size:
        #     fine_tune_images = fine_tune_images[1:]
        #     fine_tune_labels = fine_tune_labels[1:]
        if len(self.fine_tune_images) > self.max_training_set_size:
            self.fine_tune_images = self.fine_tune_images[1:]
            self.fine_tune_labels = self.fine_tune_labels[1:]
        ########################################################################

        # keep track of how often selected prey is nearest center:
        temp = xy3.copy()  # needed?
        sorted_by_dist_to_center = sort_xy3_by_proximity_to_point(temp, [0.5, 0.5])
        if sorted_by_dist_to_center[0] == sorted_xy3[0]:
            global nearest_center
            ####################################################################
            # TODO 20220729
            # nearest_center += 1
            self.nearest_center += 1
            ####################################################################
        ########################################################################
        # TODO 20220729
        # print('  nearest_center:',
        #       str(int(100 * float(nearest_center) / (step + 1))) + '%',
        #       '(nearest_center =', nearest_center, ', steps =', step + 1, ')')
        print('  nearest_center:',
              str(int(100 * float(self.nearest_center) / (step + 1))) + '%',
              '(nearest_center =', self.nearest_center, ', steps =', step + 1, ')')
        ########################################################################

        # Convert training data list to np arrays
        ########################################################################
        # TODO 20220729
        # images_array = np.array(fine_tune_images)
        # labels_array = np.array([x[0] for x in fine_tune_labels])
        images_array = np.array(self.fine_tune_images)
        labels_array = np.array([x[0] for x in self.fine_tune_labels])
        ########################################################################

        ########################################################################
        # TODO 20220730
        # print('images_array.shape =', images_array.shape)
        # print('labels_array.shape =', labels_array.shape)

        print('images_array.shape =', images_array.shape,
              '-- labels_array.shape =', labels_array.shape)

        ########################################################################

    	# skip fine-tuning until dataset is large enough (10% of max size).
        ########################################################################
        # TODO 20220729
        # if images_array.shape[0] > (max_training_set_size * 0.1):
        #     # Do fine-tuning training step using data accumulated during run.
        #     history = model.fit(x=images_array, y=labels_array)
        #     # Keep log of in_disk metric:
        #     write_in_disk_log(step, history)
        if images_array.shape[0] > (self.max_training_set_size * 0.1):
            # Do fine-tuning training step using data accumulated during run.
            history = self.model.fit(x=images_array, y=labels_array)
            # Keep log of in_disk metric:
            write_in_disk_log(step, history)
        ########################################################################

    # TODO 20220730 try running second model
    def jiggle_model(self):
        weight_perturbation(self.model)


    ################################################################################

################################################################################
# TODO 20220730
# Utility from https://stackoverflow.com/a/64542651/1991373

def weight_perturbation(model):
    """Add noise to all weights in a Keras model."""
    for layer in model.layers:
        trainable_weights = layer.trainable_variables
        for weight in trainable_weights:
            random_weights = tf.random.uniform(tf.shape(weight),
                                               # 1e-4, 1e-5,
                                               0, 0.001,
                                               dtype=tf.float32)
            weight.assign_add(random_weights)
################################################################################


# TODO 20220729 prototype storing what had been the global predator model with
# one stored in Predator class.

test_predator = Predator()
test_predator.initialize_to_pre_trained_model()

# TODO 20220730 try running second model
second_predator = Predator()
second_predator.initialize_to_pre_trained_model()
second_predator.jiggle_model()


################################################################################

Reading pre-trained model from: /content/drive/My Drive/PredatorEye/saved_models/20220321_1711_FCD6_rc4
Reading pre-trained model from: /content/drive/My Drive/PredatorEye/saved_models/20220321_1711_FCD6_rc4


# Run test

In [ ]:
################################################################################
# TODO 20220728 refactoring for Predator class.

# Keep track of how often selected prey is nearest center:
# nearest_center = 0
Predator.nearest_center = 0
# Predator.population = []

# TODO maybe a reset() method?
################################################################################


# Flush out obsolete files in comms directory.
clean_up_communication_directory()

# Start fresh run defaulting to step 0.
start_run()

Start run in /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/
Waiting for camo_0.png and prey_0.txt ... done, elapsed time: 36 seconds.
Wrote '0.35675594 0.6136136' to response file find_0.txt
"other" prediction: [0.35496944 0.6173679 ]
  nearest_center: 0% (nearest_center = 0 , steps = 1 )
images_array.shape = (1, 128, 128, 3) , labels_array.shape = (1, 2)
Waiting for camo_1.png and prey_1.txt ... done, elapsed time: 18 seconds.
Wrote '0.5495125 0.34157526' to response file find_1.txt
"other" prediction: [0.5503588  0.34200406]
  nearest_center: 50% (nearest_center = 1 , steps = 2 )
images_array.shape = (2, 128, 128, 3) , labels_array.shape = (2, 2)
Waiting for camo_2.png and prey_2.txt ... done, elapsed time: 40 seconds.
Wrote '0.7734213 0.33669877' to response file find_2.txt
"other" prediction: [0.7857189  0.33577794]
  nearest_center: 33% (nearest_center = 1 , steps = 3 )
images_array.shape = (3, 128, 128, 3) , labels_array.shape = (3, 2)
Waiting for camo_3.png and prey

In [ ]:
# Normally start from step 0, or if an "other" file exists
# (eg 'camo_123.jpeg') then restart from that point.

# restart_run()